In [40]:
import json
from collections import OrderedDict
import os

building_name = 'ebu3b'
with open("metadata/%s_label_dict.json"%building_name, 'r') as fp:
    label_dict = json.load(fp)
with open("metadata/%s_sentence_dict.json"%building_name, "r") as fp:
    sentence_dict = OrderedDict(json.load(fp))

In [42]:
with open("metadata/%s_mallet_learning_sample.txt"%building_name, "w") as fp:
    for srcid, labels in label_dict.items():
        sentence = sentence_dict[srcid]
        sentence = ["newline" if word=="\n" else word for word in sentence]
        #print "EOB"
        assert(len(sentence)==len(labels))
        for i, (word,label) in enumerate(zip(sentence,labels)):
            if i==0:
                feat_prev_word= "prev:BOD"
            else:
                feat_prev_word = "prev:%s"%sentence[i-1].lower()
            feat_curr_word = "curr:%s"%word.lower()
            if i==len(sentence)-1:
                feat_next_word = "next:EOD"
            else:
                feat_next_word = "next:%s"%sentence[i+1].lower()
            #feat_curr_label = "curr_label:%s"%label
            fp.write("{0} {1} {2} {3}\n".format(feat_prev_word, feat_curr_word, feat_next_word, label))

In [ ]:
os.system('java -cp "/home/jbkoh/tools/mallet-2.0.8/class:/home/jbkoh/tools/mallet-2.0.8/lib/mallet-deps.jar" cc.mallet.fst.SimpleTagger --train true --model-file model/{0}crf --threads 16 metadata/{0}_mallet_learning_sample.txt'.format(building_name))

In [21]:
with open('metadata/%s_mallet_test_sample.txt'%building_name, "w") as fp:
    for srcid, sentence in sentence_dict.items():
        sentence = ["newline" if word=="\n" else word for word in sentence]
        for i, word in enumerate(sentence):
            if i==0:
                feat_prev_word= "prev:BOD"
            else:
                feat_prev_word = "prev:%s"%sentence[i-1].lower()
            feat_curr_word = "curr:%s"%word.lower()
            if i==len(sentence)-1:
                feat_next_word = "next:EOD"
            else:
                feat_next_word = "next:%s"%sentence[i+1].lower()
            #feat_curr_label = "curr_label:%s"%label
            fp.write("{0} {1} {2}\n".format(feat_prev_word, feat_curr_word, feat_next_word))

In [26]:
os.system('java -Xmx8g -cp "/home/jbkoh/tools/mallet-2.0.8/class:/home/jbkoh/tools/mallet-2.0.8/lib/mallet-deps.jar" cc.mallet.fst.SimpleTagger --model-file model/{0}crf --threads 8 metadata/{0}_mallet_test_sample.txt > test_result/{0}_test.txt'.format(building_name))

0

In [29]:
with open('test_result/%s_test.txt'%building_name, 'r') as fp:
    word_label_dict = dict()
    for srcid, sentence in sentence_dict.items():
        word_label_list = list()
        for word in sentence:
            label = fp.readline()
            word_label_list.append((word, label))
        word_label_dict[srcid] = word_label_list

In [39]:
with open('test_result/%s_with_tags.txt'%building_name, 'w') as fp:
    for srcid, word_label_list in word_label_dict.items():
        

[(u'nae', 'network_adapter \n'),
 (u'06', 'identifier \n'),
 (u'n', 'network_adapter \n'),
 (u'2', 'identifier \n'),
 (u'2', 'identifier \n'),
 (u'vma', 'vav \n'),
 (u'181', 'identifier \n'),
 (u'htgpid', 'heating_command \n'),
 (u'o', 'none \n'),
 (u'\n', 'none \n'),
 (u'ebu', 'building \n'),
 (u'3', 'identifier \n'),
 (u'b', 'identifier \n'),
 (u'.', 'none \n'),
 (u'rm', 'room \n'),
 (u'-', 'none \n'),
 (u'4128', 'identifier \n'),
 (u'.', 'none \n'),
 (u'.', 'none \n'),
 (u'htgpid', 'heating_command \n'),
 (u'-', 'none \n'),
 (u'o', 'none \n'),
 (u'\n', 'none \n'),
 (u'heating', 'heating \n'),
 (u'command', 'command \n'),
 (u'\n', 'none \n')]